In [1]:
import pandas as pd
import re 
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
%matplotlib inline

pd.set_option("display.max_columns",30)
pd.set_option("display.max_colwidth",100)
pd.set_option("display.precision",3)

# Use the file location of your Import.io csv
CSV_PATH=r"H:/jupyter/machinelearning/PythonMachineLearningBlueprints/PMLB Datasets copy/magic.csv"

df=pd.read_csv(CSV_PATH)
df.columns  #输出列标题

Index(['routablemask_link', 'routablemask_link/_text',
       'routablemask_link/_title', 'routablemask_link_numbers',
       'routablemask_content', 'imagebadge_value', 'imagebadge_value_numbers',
       'routable_link', 'routable_link/_text', 'routable_link/_title',
       'routable_link_numbers', 'listingtype_value', 'pricelarge_value',
       'pricelarge_value_prices', 'propertyinfo_value',
       'propertyinfo_value_numbers', 'fineprint_value',
       'fineprint_value_numbers', 'tozcount_number', 'tozfresh_value',
       'tablegrouped_values', 'tablegrouped_values_prices', '_PAGE_NUMBER'],
      dtype='object')

In [4]:
df.head().T    #T 转置

,0,1,2,3,4
routablemask_link,"http://www.zillow.com/b/2-Ellwood-St-New-York-NY/40.85899,-73.92814_ll/","http://www.zillow.com/b/603-W-140th-St-New-York-NY/40.82376,-73.95335_ll/",http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT-2C-New-York-NY-10031/124451755_zpid/,http://www.zillow.com/homedetails/413-E-84th-St-APT-8-New-York-NY-10028/2100761260_zpid/
routablemask_link/_text,5 photos,10 photos,NaN,9 photos,5 photos
routablemask_link/_title,NaN,NaN,NaN,NaN,NaN
routablemask_link_numbers,5,10,NaN,9,5
routablemask_content,NaN,NaN,NaN,NaN,NaN
imagebadge_value,5 photos,10 photos,NaN,9 photos,5 photos
imagebadge_value_numbers,5,10,NaN,9,5
routable_link,"http://www.zillow.com/b/2-Ellwood-St-New-York-NY/40.85899,-73.92814_ll/","http://www.zillow.com/b/603-W-140th-St-New-York-NY/40.82376,-73.95335_ll/",http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT-2C-New-York-NY-10031/124451755_zpid/,http://www.zillow.com/homedetails/413-E-84th-St-APT-8-New-York-NY-10028/2100761260_zpid/
routable_link/_text,2 Ellwood St,603 W 140th St,"9 E 129th St # 1, New York, NY10035","710 Riverside Dr APT 2C, New York, NY10031","413 E 84th St APT 8, New York, NY10028"
routable_link/_title,"2 Ellwood St APT 5H, New York, NY Real Estate","603 W 140th St APT 44, New York, NY Real Estate","9 E 129th St # 1, New York, NY Real Estate","710 Riverside Dr APT 2C, New York, NY Real Estate","413 E 84th St APT 8, New York, NY Real Estate"


In [9]:
# multiple units
mu=df[df['listingtype_value'].str.contains('Apartments For')]

# single units
su=df[df['listingtype_value'].str.contains('Apartment For')]

len(mu)

161

In [11]:
len(su)

339

In [14]:
len(su[su['pricelarge_value_prices'].isnull()])

0

In [12]:
su['propertyinfo_value']

2                                               1 bd • 1 ba
3                                 2 bds • 2 ba • 1,016 sqft
4                                             Studio • 1 ba
5                                              2 bds • 1 ba
7                                              2 bds • 1 ba
10                                              1 bd • 1 ba
11     1 bd • 1 ba • 496 sqft • 4,502 sqft lot • Built 1900
12                                            Studio • 1 ba
13                                              1 bd • 1 ba
17                                              1 bd • 1 ba
18                                            Studio • 1 ba
19                                              1 bd • 1 ba
20                                   1 bd • 1 ba • 700 sqft
21                                              1 bd • 1 ba
23                                 Studio • 1 ba • 487 sqft
24                                             2 bds • 1 ba
27                                   1 b

In [15]:
# 检查没有包含‘bd’或 'Studio'的行数
len(su[~(su['propertyinfo_value'].str.contains('Studio')|su['propertyinfo_value'].str.contains('bd'))])

0

In [20]:
# 检查没有包含‘ba’的行数
len(su[~(su['propertyinfo_value'].str.contains('ba'))])

6

In [29]:
# 选择拥有浴室的房源
no_baths = su[~(su['propertyinfo_value'].str.contains('ba'))]

# 排除缺失了浴室的房源
sucln=su[~su.index.isin(no_baths.index)]

# 使用项目符号进行切分
def parse_info(row):
        if not 'sqft' in row:
            br, ba = row.split('•')[:2]
            sqft = np.nan
        else:
            br, ba, sqft = row.split('•')[:3]                
        return pd.Series({'Beds': br, 'Baths': ba, 'Sqft': sqft})

attr=sucln['propertyinfo_value'].apply(parse_info)

attr

,Baths,Beds,Sqft
2,1 ba,1 bd,NaN
3,2 ba,2 bds,"1,016 sqft"
4,1 ba,Studio,NaN
5,1 ba,2 bds,NaN
7,1 ba,2 bds,NaN
10,1 ba,1 bd,NaN
11,1 ba,1 bd,496 sqft
12,1 ba,Studio,NaN
13,1 ba,1 bd,NaN
17,1 ba,1 bd,NaN


In [30]:
#在取值中将字符串删除
attr_cln = attr.applymap(lambda x: x.strip().split(' ')[0] if isinstance(x,str) else np.nan)

attr_cln

,Baths,Beds,Sqft
2,1,1,NaN
3,2,2,"1,016"
4,1,Studio,NaN
5,1,2,NaN
7,1,2,NaN
10,1,1,NaN
11,1,1,496
12,1,Studio,NaN
13,1,1,NaN
17,1,1,NaN


In [31]:
sujnd=sucln.join(attr_cln)
sujnd.T

,2,3,4,5,7,10,11,12,13,17,18,19,20,21,23,...,470,471,475,477,479,483,484,485,486,487,490,492,496,497,499
routablemask_link,http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT-2C-New-York-NY-10031/124451755_zpid/,http://www.zillow.com/homedetails/413-E-84th-St-APT-8-New-York-NY-10028/2100761260_zpid/,http://www.zillow.com/homedetails/324-Pleasant-Ave-APT-6E-New-York-NY-10035/2100761258_zpid/,http://www.zillow.com/homedetails/217-Thompson-St-APT-6-New-York-NY-10012/2100761371_zpid/,http://www.zillow.com/homedetails/340-W-47th-St-APT-4C-New-York-NY-10036/2108861613_zpid/,http://www.zillow.com/homedetails/1356-1st-Ave-APT-5B-New-York-NY-10021/31539199_zpid/,http://www.zillow.com/homedetails/101-W-77th-St-APT-3C-New-York-NY-10024/2114511658_zpid/,http://www.zillow.com/homedetails/111-W-74th-St-APT-2R-New-York-NY-10023/2123304010_zpid/,http://www.zillow.com/homedetails/646-10th-Ave-APT-4B-New-York-NY-10036/2114397534_zpid/,http://www.zillow.com/homedetails/76-Carmine-St-APT-4C-New-York-NY-10014/2110813540_zpid/,http://www.zillow.com/homedetails/146-W-120th-St-APT-8-New-York-NY-10027/2100762986_zpid/,http://www.zillow.com/homedetails/137-W-72nd-St-APT-4FE-New-York-NY-10023/2105480982_zpid/,http://www.zillow.com/homedetails/178-Avenue-B-2-New-York-NY-10009/2100763265_zpid/,http://www.zillow.com/homedetails/95-Wall-St-72791-New-York-NY-10005/2103168440_zpid/,...,http://www.zillow.com/homedetails/304-E-41st-St-APT-1107A-New-York-NY-10017/2100774366_zpid/,http://www.zillow.com/homedetails/169-Avenue-A-APT-1-New-York-NY-10009/2105208117_zpid/,http://www.zillow.com/homedetails/722-10th-Ave-APT-3A-New-York-NY-10019/2126268388_zpid/,http://www.zillow.com/homedetails/200-W-70th-St-APT-11L3-New-York-NY-10023/2100774851_zpid/,http://www.zillow.com/homedetails/219-W-78th-St-APT-2F-New-York-NY-10024/2100775303_zpid/,http://www.zillow.com/homedetails/405-W-49th-St-APT-2W-New-York-NY-10019/2100776286_zpid/,http://www.zillow.com/homedetails/230-E-26th-St-APT-3A-New-York-NY-10010/2100776283_zpid/,http://www.zillow.com/homedetails/319-E-53rd-St-APT-2B-New-York-NY-10022/2104750345_zpid/,http://www.zillow.com/homedetails/53-Irving-Pl-APT-3K-New-York-NY-10003/2103128441_zpid/,http://www.zillow.com/homedetails/25-E-67th-St-APT-8D-New-York-NY-10065/2100777265_zpid/,http://www.zillow.com/homedetails/303-E-73rd-St-E-New-York-NY-10021/2100777278_zpid/,http://www.zillow.com/homedetails/422-E-81st-St-APT-4A-New-York-NY-10028/2111603077_zpid/,http://www.zillow.com/homedetails/445-W-49th-St-APT-3B-New-York-NY-10019/2106388119_zpid/,http://www.zillow.com/homedetails/203-E-121st-St-APT-901-New-York-NY-10035/2100777283_zpid/,http://www.zillow.com/homedetails/434-E-72nd-St-APT-2C-New-York-NY-10021/2120058527_zpid/
routablemask_link/_text,NaN,9 photos,5 photos,7 photos,NaN,NaN,9 photos,4 photos,6 photos,NaN,4 photos,6 photos,10 photos,6 photos,17 photos,...,5 photos,8 photos,5 photos,4 photos,6 photos,5 photos,6 photos,5 photos,6 photos,NaN,5 photos,4 photos,4 photos,4 photos,1 photo
routablemask_link/_title,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
routablemask_link_numbers,NaN,9,5,7,NaN,NaN,9,4,6,NaN,4,6,10,6,17,...,5,8,5,4,6,5,6,5,6,NaN,5,4,4,4,1
routablemask_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
imagebadge_value,NaN,9 photos,5 photos,7 photos,NaN,NaN,9 photos,4 photos,6 photos,NaN,4 photos,6 photos,10 photos,6 photos,17 photos,...,5 photos,8 photos,5 photos,4 photos,6 photos,5 photos,6 photos,5 photos,6 photos,NaN,5 photos,4 photos,4 photos,4 photos,1 photo
imagebadge_value_numbers,NaN,9,5,7,NaN,NaN,9,4,6,NaN,4,6,10,6,17,...,5,8,5,4,6,5,6,5,6,NaN,5,4,4,4,1
routable_link,http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT

In [33]:
# parse out zip, floor

def parse_addy(r):
    so_zip = re.search(', NY(\d+)', r)
    so_flr = re.search('(?:APT|#)\s+(\d+)[A-Z]+,', r)
    if so_zip:
        zipc = so_zip.group(1)
    else:
        zipc = np.nan
    if so_flr:
        flr = so_flr.group(1)
    else:
        flr = np.nan
    return pd.Series({'Zip':zipc, 'Floor': flr})

flrzip = sujnd['routable_link/_text'].apply(parse_addy)
flrzip

,Floor,Zip
2,NaN,10035
3,2,10031
4,NaN,10028
5,6,10035
7,NaN,10012
10,4,10036
11,5,10021
12,3,10024
13,2,10023
17,4,10036


In [34]:
len(flrzip[~flrzip['Floor'].isnull()])

164

In [35]:
len(flrzip[~flrzip['Zip'].isnull()])

320

In [36]:
suf = sujnd.join(flrzip)
suf.T

,2,3,4,5,7,10,11,12,13,17,18,19,20,21,23,...,470,471,475,477,479,483,484,485,486,487,490,492,496,497,499
routablemask_link,http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT-2C-New-York-NY-10031/124451755_zpid/,http://www.zillow.com/homedetails/413-E-84th-St-APT-8-New-York-NY-10028/2100761260_zpid/,http://www.zillow.com/homedetails/324-Pleasant-Ave-APT-6E-New-York-NY-10035/2100761258_zpid/,http://www.zillow.com/homedetails/217-Thompson-St-APT-6-New-York-NY-10012/2100761371_zpid/,http://www.zillow.com/homedetails/340-W-47th-St-APT-4C-New-York-NY-10036/2108861613_zpid/,http://www.zillow.com/homedetails/1356-1st-Ave-APT-5B-New-York-NY-10021/31539199_zpid/,http://www.zillow.com/homedetails/101-W-77th-St-APT-3C-New-York-NY-10024/2114511658_zpid/,http://www.zillow.com/homedetails/111-W-74th-St-APT-2R-New-York-NY-10023/2123304010_zpid/,http://www.zillow.com/homedetails/646-10th-Ave-APT-4B-New-York-NY-10036/2114397534_zpid/,http://www.zillow.com/homedetails/76-Carmine-St-APT-4C-New-York-NY-10014/2110813540_zpid/,http://www.zillow.com/homedetails/146-W-120th-St-APT-8-New-York-NY-10027/2100762986_zpid/,http://www.zillow.com/homedetails/137-W-72nd-St-APT-4FE-New-York-NY-10023/2105480982_zpid/,http://www.zillow.com/homedetails/178-Avenue-B-2-New-York-NY-10009/2100763265_zpid/,http://www.zillow.com/homedetails/95-Wall-St-72791-New-York-NY-10005/2103168440_zpid/,...,http://www.zillow.com/homedetails/304-E-41st-St-APT-1107A-New-York-NY-10017/2100774366_zpid/,http://www.zillow.com/homedetails/169-Avenue-A-APT-1-New-York-NY-10009/2105208117_zpid/,http://www.zillow.com/homedetails/722-10th-Ave-APT-3A-New-York-NY-10019/2126268388_zpid/,http://www.zillow.com/homedetails/200-W-70th-St-APT-11L3-New-York-NY-10023/2100774851_zpid/,http://www.zillow.com/homedetails/219-W-78th-St-APT-2F-New-York-NY-10024/2100775303_zpid/,http://www.zillow.com/homedetails/405-W-49th-St-APT-2W-New-York-NY-10019/2100776286_zpid/,http://www.zillow.com/homedetails/230-E-26th-St-APT-3A-New-York-NY-10010/2100776283_zpid/,http://www.zillow.com/homedetails/319-E-53rd-St-APT-2B-New-York-NY-10022/2104750345_zpid/,http://www.zillow.com/homedetails/53-Irving-Pl-APT-3K-New-York-NY-10003/2103128441_zpid/,http://www.zillow.com/homedetails/25-E-67th-St-APT-8D-New-York-NY-10065/2100777265_zpid/,http://www.zillow.com/homedetails/303-E-73rd-St-E-New-York-NY-10021/2100777278_zpid/,http://www.zillow.com/homedetails/422-E-81st-St-APT-4A-New-York-NY-10028/2111603077_zpid/,http://www.zillow.com/homedetails/445-W-49th-St-APT-3B-New-York-NY-10019/2106388119_zpid/,http://www.zillow.com/homedetails/203-E-121st-St-APT-901-New-York-NY-10035/2100777283_zpid/,http://www.zillow.com/homedetails/434-E-72nd-St-APT-2C-New-York-NY-10021/2120058527_zpid/
routablemask_link/_text,NaN,9 photos,5 photos,7 photos,NaN,NaN,9 photos,4 photos,6 photos,NaN,4 photos,6 photos,10 photos,6 photos,17 photos,...,5 photos,8 photos,5 photos,4 photos,6 photos,5 photos,6 photos,5 photos,6 photos,NaN,5 photos,4 photos,4 photos,4 photos,1 photo
routablemask_link/_title,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
routablemask_link_numbers,NaN,9,5,7,NaN,NaN,9,4,6,NaN,4,6,10,6,17,...,5,8,5,4,6,5,6,5,6,NaN,5,4,4,4,1
routablemask_content,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
imagebadge_value,NaN,9 photos,5 photos,7 photos,NaN,NaN,9 photos,4 photos,6 photos,NaN,4 photos,6 photos,10 photos,6 photos,17 photos,...,5 photos,8 photos,5 photos,4 photos,6 photos,5 photos,6 photos,5 photos,6 photos,NaN,5 photos,4 photos,4 photos,4 photos,1 photo
imagebadge_value_numbers,NaN,9,5,7,NaN,NaN,9,4,6,NaN,4,6,10,6,17,...,5,8,5,4,6,5,6,5,6,NaN,5,4,4,4,1
routable_link,http://www.zillow.com/homedetails/9-E-129th-St-1-New-York-NY-10035/2100761096_zpid/,http://www.zillow.com/homedetails/710-Riverside-Dr-APT

In [38]:
# we'll reduce the data down to the columns of interest
sudf = suf[['pricelarge_value_prices','Beds','Baths','Sqft','Floor','Zip']]

In [39]:
# we'll also clean up the weird column name
sudf.rename(columns={'pricelarge_value_prices':'Rent'}, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [40]:
sudf

,Rent,Beds,Baths,Sqft,Floor,Zip
2,1750.0,1,1,NaN,NaN,10035
3,3000.0,2,2,"1,016",2,10031
4,2300.0,Studio,1,NaN,NaN,10028
5,2500.0,2,1,NaN,6,10035
7,2800.0,2,1,NaN,NaN,10012
10,2490.0,1,1,NaN,4,10036
11,2750.0,1,1,496,5,10021
12,2150.0,Studio,1,NaN,3,10024
13,2875.0,1,1,NaN,2,10023
17,2225.0,1,1,NaN,4,10036


In [41]:
sudf.rename(columns={'Rent':'租金','Beds':'卧室','Baths':'浴室','Sqft':'平方英尺','Floor':'楼层','Zip':'邮政编码'}, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [42]:
sudf

,租金,卧室,浴室,平方英尺,楼层,邮政编码
2,1750.0,1,1,NaN,NaN,10035
3,3000.0,2,2,"1,016",2,10031
4,2300.0,Studio,1,NaN,NaN,10028
5,2500.0,2,1,NaN,6,10035
7,2800.0,2,1,NaN,NaN,10012
10,2490.0,1,1,NaN,4,10036
11,2750.0,1,1,496,5,10021
12,2150.0,Studio,1,NaN,3,10024
13,2875.0,1,1,NaN,2,10023
17,2225.0,1,1,NaN,4,10036


In [43]:
# 重新编号
sudf.reset_index(drop=True,inplace=True)
sudf

,租金,卧室,浴室,平方英尺,楼层,邮政编码
0,1750.0,1,1,NaN,NaN,10035
1,3000.0,2,2,"1,016",2,10031
2,2300.0,Studio,1,NaN,NaN,10028
3,2500.0,2,1,NaN,6,10035
4,2800.0,2,1,NaN,NaN,10012
5,2490.0,1,1,NaN,4,10036
6,2750.0,1,1,496,5,10021
7,2150.0,Studio,1,NaN,3,10024
8,2875.0,1,1,NaN,2,10023
9,2225.0,1,1,NaN,4,10036


In [44]:
sudf.describe()

,租金
count,333.000
mean,2492.628
std,366.882
min,1500.000
25%,2200.000
50%,2525.000
75%,2800.000
max,3000.000


In [46]:
# we'll replace 'Studio' with 0 where present
sudf.loc[:,'卧室'] = sudf['卧室'].map(lambda x: 0 if 'Studio' in x else x)
sudf

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,租金,卧室,浴室,平方英尺,楼层,邮政编码
0,1750.0,1,1,NaN,NaN,10035
1,3000.0,2,2,"1,016",2,10031
2,2300.0,0,1,NaN,NaN,10028
3,2500.0,2,1,NaN,6,10035
4,2800.0,2,1,NaN,NaN,10012
5,2490.0,1,1,NaN,4,10036
6,2750.0,1,1,496,5,10021
7,2150.0,0,1,NaN,3,10024
8,2875.0,1,1,NaN,2,10023
9,2225.0,1,1,NaN,4,10036


In [47]:
# we're not getting stats on 'Beds','Baths', and 'Floor' because of the datatype
sudf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 6 columns):
租金      333 non-null float64
卧室      333 non-null object
浴室      333 non-null object
平方英尺    108 non-null object
楼层      164 non-null object
邮政编码    320 non-null object
dtypes: float64(1), object(5)
memory usage: 15.7+ KB


In [48]:
sudf.rename(columns={'租金':'Rent','卧室':'Beds','浴室':'Baths','平方英尺':'Sqft','楼层':'Floor','邮政编码':'Zip'}, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [49]:
# 修改每一列的数据类型
# let's fix the datatype for the columns
sudf.loc[:,'Rent'] = sudf['Rent'].astype(int)
sudf.loc[:,'Beds'] = sudf['Beds'].astype(int)

# half baths require a float
sudf.loc[:,'Baths'] = sudf['Baths'].astype(float)

# with NaNs we need float, but we have to replace commas first
sudf.loc[:,'Sqft'] = sudf['Sqft'].str.replace(',','')

sudf.loc[:,'Sqft'] = sudf['Sqft'].astype(float)
sudf.loc[:,'Floor'] = sudf['Floor'].astype(float)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [50]:
sudf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 6 columns):
Rent     333 non-null int32
Beds     333 non-null int32
Baths    333 non-null float64
Sqft     108 non-null float64
Floor    164 non-null float64
Zip      320 non-null object
dtypes: float64(3), int32(2), object(1)
memory usage: 13.1+ KB


In [51]:
sudf.describe()

,Rent,Beds,Baths,Sqft,Floor
count,333.000,333.000,333.000,108.000,164.000
mean,2492.628,0.820,1.008,528.981,11.201
std,366.882,0.718,0.082,133.050,86.178
min,1500.000,0.000,1.000,280.000,1.000
25%,2200.000,0.000,1.000,447.500,2.000
50%,2525.000,1.000,1.000,512.000,4.000
75%,2800.000,1.000,1.000,600.000,5.000
max,3000.000,3.000,2.000,1090.000,1107.000


In [52]:
sudf[sudf['Floor']>5]

,Rent,Beds,Baths,Sqft,Floor,Zip
3,2500,2,1.0,NaN,6.0,10035
29,1815,1,1.0,596.0,6.0,10037
43,1815,1,1.0,596.0,6.0,10037
44,2950,0,1.0,NaN,7.0,10036
59,3000,0,1.0,541.0,9.0,10013
74,2295,1,1.0,NaN,11.0,10002
77,2999,1,1.0,NaN,12.0,10012
87,2350,2,1.0,550.0,6.0,10003
93,2550,0,1.0,425.0,14.0,10006
110,2400,0,1.0,550.0,8.0,10002


In [53]:
sudf=sudf.drop([318])

In [54]:
sudf.describe()

,Rent,Beds,Baths,Sqft,Floor
count,332.000,332.000,332.000,108.000,163.000
mean,2493.509,0.822,1.008,528.981,4.479
std,367.083,0.718,0.082,133.050,3.861
min,1500.000,0.000,1.000,280.000,1.000
25%,2200.000,0.000,1.000,447.500,2.000
50%,2527.500,1.000,1.000,512.000,4.000
75%,2800.000,1.000,1.000,600.000,5.000
max,3000.000,3.000,2.000,1090.000,32.000


In [55]:
sudf.pivot_table('Rent','Zip','Beds',aggfunc='mean')

Beds,0,1,2,3
Zip,,,,
10001,2737.500,NaN,NaN,NaN
10002,2283.438,2422.513,2792.647,NaN
10003,2109.889,2487.812,2525.000,NaN
10004,2798.750,2850.000,NaN,NaN
10005,2516.000,NaN,NaN,NaN
10006,2611.000,2788.000,NaN,NaN
10009,2200.909,2568.441,2530.000,NaN
10010,NaN,2299.000,2940.000,NaN
10011,2774.667,2852.000,2595.000,NaN


In [ ]:
su_lt_two=sudf[sudf['Beds']<2]

In [ ]:
import folium

map = folium.Map(location=[40.748817, -73.985428], zoom_start=13)
map.geo_json(geo_path=r'C:/Users/Administrator/Desktop/nyc.geojson', data=su_lt_two,
             columns=['Zip', 'Rent'],
             key_on='feature.properties.postalCode',
             threshold_scale=[1700.00, 1900.00, 2100.00, 2300.00, 2500.00, 2750.00],
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Rent (%)',
              reset=True)
map.create_map(path='nyc.html')

sudf